In [86]:
# Importing the required libraries

#import community as community_louvain

import numpy as np
import networkx as nx
import scipy as sp

import networkx.algorithms.community as nx_comm
%matplotlib inline


In [87]:
np.seterr(invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [88]:
NODES_FILE = './nodes_repo_Com_FFForks.csv'

In [90]:
G = nx.read_edgelist('./forks_final_all.csv',
                     create_using=nx.MultiDiGraph(), delimiter = ', ')

In [117]:
#sets communities node attribute to node
nodesFile = open(NODES_FILE, 'r')
record = {}
# skip headers
#nodesFile.readline()

# store nodes in record
for line in nodesFile:
    
    toRecord = line.split(", ")
    #print(toRecord)
    if (toRecord[3] != '-1 \n'):
        record[toRecord[0]] = int(toRecord[3].strip("\n"))
nx.set_node_attributes(G,record,"communities")   




In [118]:
#create adj matrix
SP_adj_matrix = nx.adjacency_matrix(G)
NP_adj_matrix = SP_adj_matrix.todense()


In [119]:
#create corresponding list with community that node is affiliated with
community_affiliation = []
for i in range(len(NP_adj_matrix)):
    community_affiliation.append(list(G.nodes("communities"))[i][1])


FUNCTION BELOW IS MODIFIED BUT TAKEN FROM: https://github.com/CarloNicolini/pyconnectivity/blob/master/bct.py

In [99]:
def participation_coef(W, ci):
    '''
    Participation coefficient is a measure of diversity of intermodular
    connections of individual nodes.
    Inputs:     W,      binary/weighted, directed/undirected connection matrix
               Ci,         community affiliation vector
    Output:     P,      participation coefficient
    Note: The output for directed graphs is the "out-neighbor"
         participation coefficient.
    '''
  

    _, ci = np.unique(ci, return_inverse=True)
    ci += 1
   
    n = len(W)  # number of vertices
    Ko = np.sum(W, axis=1)  # (out) degree
    
    Gc = np.dot((W != 0), np.diag(ci))  # neighbor community affiliation
    
    Kc2 = np.zeros((n,1))  # community-specific neighbors
   
    for i in range(int(np.max(ci))):
        Kc2 += np.sqrt(np.sum(W * (Gc == i), axis=1)) #add more sqrt if necessary (if values still negative)
        checker,_ = np.where(Kc2<0) #check to see if any values of Kc overflow and become negative
        if(len(checker)>0):
            print("OVERFLOW. TRY AGAIN WITH MORE SQRT")
            break
        if(i % 5 == 0 or i==(int(np.max(ci)))-1):
            print(str(i) + " out of " + str(np.max(ci)-1) + " communities complete")
        
    non_normal = Kc2 / np.sqrt(Ko) 
    non_normal = np.nan_to_num(non_normal,nan=np.nanmax(non_normal)) #change any nan values to max value in vector
    normalized = (non_normal - non_normal.min()) / (non_normal.max() - non_normal.min()) #normalized vector
    P = np.ones((n,1)) - normalized
    
    #P=0 if for nodes with no (out) neighbors
    P[np.where(np.logical_not(Ko))] = 0

    return P

In [100]:
Pc = participation_coef(NP_adj_matrix,community_affiliation)

0 out of 389 communities complete
5 out of 389 communities complete
10 out of 389 communities complete
15 out of 389 communities complete
20 out of 389 communities complete
25 out of 389 communities complete
30 out of 389 communities complete
35 out of 389 communities complete
40 out of 389 communities complete
45 out of 389 communities complete
50 out of 389 communities complete
55 out of 389 communities complete
60 out of 389 communities complete
65 out of 389 communities complete
70 out of 389 communities complete
75 out of 389 communities complete
80 out of 389 communities complete
85 out of 389 communities complete
90 out of 389 communities complete
95 out of 389 communities complete
100 out of 389 communities complete
105 out of 389 communities complete
110 out of 389 communities complete
115 out of 389 communities complete
120 out of 389 communities complete
125 out of 389 communities complete
130 out of 389 communities complete
135 out of 389 communities complete
140 out of 389

In [102]:
print(Pc)

[[0.52506463]
 [0.75835254]
 [0.65509551]
 ...
 [0.94576383]
 [0.        ]
 [0.95295424]]


In [103]:
#find avg participation coefficient of the whole network
avg_pc = np.mean(Pc)
print(avg_pc)

0.7211734798588747


In [104]:
#find max and min average participation coefficient
#list size is amount of communities
community_pc = [0] * int(np.max(community_affiliation)+1)
#print(Pc)
#print(Pc.item(0))
#print(len(community_pc))
#print(community_affiliation)
#number of nodes in each community to calc average
num_of_nodes_in_comm = [0] * int(np.max(community_affiliation)+1) #size of 423
#print(len(NP_adj_matrix))
#For each community find the average PC
for i in range(int(np.max(community_affiliation))+1): 
    #for each node get their PC 
    for j in range(len(NP_adj_matrix)):
        #if node is in community i 
        if(community_affiliation[j] == i):
            #then add PC to community_pc
            community_pc[i] += Pc.item(j)
            #add 1 to index 
            num_of_nodes_in_comm[community_affiliation[j]] += 1      
print(community_pc)
print(num_of_nodes_in_comm)            
avg_community_pc = [0] * int(np.max(community_affiliation)+1) #423

#average out each element of avg_community_pc
for i in range(len(avg_community_pc)):
    #PC[i] / # of nodes in community igy
    avg_community_pc[i] = community_pc[i]/num_of_nodes_in_comm[i]
    

#print(avg_community_pc)
print(np.max(avg_community_pc))
print(np.min(avg_community_pc))
print(np.where(avg_community_pc ==np.max(avg_community_pc) ))
print(np.where(avg_community_pc ==np.min(avg_community_pc) ))

[4.372073977079418, 250.00010333741096, 1.8670187186178997, 1.5525415449763178, 5.660999650528439, 4.486647008060916, 0.9999999999999997, 16.24125565104466, 4.420442683335167, 5.725433509262987, 1.416257989868915, 4.205320004270745, 1.824780622329711, 0.9230277153860683, 1.631838312921623, 2.91077355799154, 2.7167460680411883, 2.8998963166297926, 3.915308509444049, 1.8867087755057483, 3.582635888376053, 2.1936595669595866, 1.9999999999999993, 1.4070372198517003, 3.8725405632643195, 4.259855131317355, 2.5099597066955295, 4.213517639166373, 1.8635924665331154, 17.729463840018077, 1.3333533618945212, 0.6774948648869388, 1.0, 5.9469624837439, 1.7207690282655954, 0.9666320794692136, 0.894196511574139, 11.991503060476054, 1.6544341583532223, 2.8588627256926595, 4.517150777915132, 1.8118175937671404, 0.9999999999999997, 0.9999999999999997, 5.866528474337803, 2.3246946749615205, 0.9999999999999997, 0.7227462778555924, 3.945436970388898, 5.0781286737926905, 1.9999999999999993, 0.999999999999999

Write PC to csv file

In [120]:
NODES_FILE1 = "nodes_communities_PC1.csv"
nodesFile1 = open(NODES_FILE1, 'r')
record1 = {}
# skip headers
#nodesFile.readline()

# store nodes in record
for line in nodesFile1:
    toRecord1 = line.split(", ")
    user1 = toRecord1[0].strip()
    repo1 = toRecord1[1].strip()
    follower_community1 = toRecord1[2].strip()
    fork_community1 = toRecord1[3].strip()
    follower_pc1 = toRecord1[4].strip()
    fork_pc1 = -1
    
    record1[toRecord1[0]] = [repo1,follower_community1,fork_community1,follower_pc1,fork_pc1]

nodesFile1.close()

In [115]:
#create dictionary with pc attached
nodes = list(G.nodes())

pc_record = {}

for i in range(len(nodes)):
    pc_record[nodes[i]] = Pc.item(i)

print(pc_record)

{'taylorotwell': 0.525064629930962, 'nunomaduro': 0.7583525415748049, 'fideloper': 0.6550955087297858, 'franzliedke': 0.8528843252177011, 'WebReflection': 0.9059085881135015, 'loic-sharma': 0.9999999999999997, 'mperham': 0.8377092882664791, 'jasonlewis': 0.9332641850097045, 'JosephSilber': 1.0, 'lucadegasperi': 0.0, 'cviebrock': 0.8207871543097718, 'machuga': 0.0, 'kirkbushell': 0.0, 'fzaninotto': 0.0, 'bryantebeek': 0.8998963166297931, 'barryvdh': 0.6719960439028374, 'rlerdorf': 0.791854973859073, 'crynobone': 0.4785845400339601, 'bobthecow': 0.8281133768581241, 'svenluijten': 0.0, 'calebporzio': 0.0, 'wesbos': 0.9999999999999997, 'daylerees': 0.0, 'tj': 0.7904738349429064, 'GrahamCampbell': 0.0, 'yajra': 0.0, 'themsaid': 0.6086257157449537, 'michaeldyrynda': 0.9454369703888994, 'JeffreyWay': 0.7577454519344969, 'pedroborges': 0.46330374797136176, 'olivierlacan': 0.8008328551306507, 'mnapoli': 0.65341383637365, 'willdurand': 0.0, 'Anahkiasen': 0.0, 'antonioribeiro': 0.6723344232317008

In [116]:
#write PC of each node into excel nodeslist
OUTPUT_NODES_FILE = './nodes_communities_PC2.csv'

counter = 0
for contributor in pc_record:

    record1[contributor][4] = Pc.item(counter)
    #print(record[contributor][3])
    counter +=1
#print(record1)

newNodesFile = open(OUTPUT_NODES_FILE, 'w+')

for user in record1:
    newNodesFile.write(f"{user}, {record1[user][0]}, {record1[user][1]}, {record1[user][2]}, {record1[user][3]}, {record1[user][4]}\n")
newNodesFile.close()